In [24]:
# Import Spacy

import spacy
from spacy import displacy
from spacy.tokens import Doc
import random
import plac
from spacy.util import minibatch
from spacy.util import compounding
from spacy.training.example import Example
from pathlib import Path




In [25]:
# nlp = spacy.load("fr_core_news_lg")
# doc = nlp("Je veux aller voir mon ami Albert à Tours en partant de Bordeaux")

# displacy.render(doc, style="dep") # (1)
# displacy.render(doc, style="ent") # (2)

In [26]:
# for token in doc:
#     print(token , token.tag_, token.dep_, token.ent_type_)
    

In [27]:
# for token in doc:
#     if token.dep_:
#         print(f"Token : {token.text}, Relation : {token.dep_}, Dépendant : {token.head.text}")

In [28]:
TRAIN_DATA = [
    ("Je voudrais aller de Paris à Marseille", {"entities": [(18, 23, "DEPART"), (27, 36, "ARRIVER")]}),
    ("Je veux prendre un train de Lyon à Bordeaux", {"entities": [(21, 25, "DEPART"), (29, 36, "ARRIVER")]}),
    ("Comment puis-je me rendre de Toulouse à Lille ?", {"entities": [(32, 40, "DEPART"), (44, 49, "ARRIVER")]}),
    ("Il y a-t-il des trains de Nice à Strasbourg ?", {"entities": [(26, 30, "DEPART"), (34, 44, "ARRIVER")]}),
    ("Pouvez-vous m'indiquer le trajet de Nantes à Bordeaux ?", {"entities": [(36, 41, "DEPART"), (45, 53, "ARRIVER")]}),
    ("Je souhaite voyager de Lyon à Paris", {"entities": [(26, 30, "DEPART"), (34, 39, "ARRIVER")]}),
    ("Y a-t-il un moyen de se rendre de Marseille à Toulouse ?", {"entities": [(29, 38, "DEPART"), (42, 50, "ARRIVER")]}),
    ("Je veux aller voir ma famille à Bordeaux en partant de Paris", {"entities": [(27, 35, "ARRIVER"), (39, 44, "DEPART")]}),
    ("Pouvez-vous me dire comment aller de Lille à Lyon ?", {"entities": [(32, 37, "DEPART"), (41, 45, "ARRIVER")]}),
    ("Il y a-t-il des trains de Strasbourg à Nice ?", {"entities": [(26, 38, "DEPART"), (42, 46, "ARRIVER")]}),
    ("Je veux aller passer mes vacances à Lille en partant de Paris", {"entities": [(27, 32, "ARRIVER"), (36, 41, "DEPART")]}),
    ("Pouvez-vous me dire comment aller de Paris à Nice ?", {"entities": [(32, 37, "DEPART"), (41, 45, "ARRIVER")]}),
    ("Il y a-t-il des trains de Toulouse à Bordeaux ?", {"entities": [(26, 34, "DEPART"), (38, 45, "ARRIVER")]}),
    ("Je souhaite voyager de Marseille à Lyon", {"entities": [(29, 38, "DEPART"), (42, 47, "ARRIVER")]}),
    ("Y a-t-il un moyen de se rendre de Bordeaux à Nice ?", {"entities": [(27, 35, "DEPART"), (39, 43, "ARRIVER")]}),
    ("Je veux aller voir ma famille à Paris en partant de Lyon", {"entities": [(27, 35, "ARRIVER"), (39, 44, "DEPART")]}),
    ("Pouvez-vous me dire comment aller de Lille à Toulouse ?", {"entities": [(32, 37, "DEPART"), (41, 49, "ARRIVER")]}),
    ("Il y a-t-il des trains de Lyon à Strasbourg ?", {"entities": [(26, 30, "DEPART"), (34, 45, "ARRIVER")]}),
    ("Je veux aller passer mes vacances à Bordeaux en partant de Marseille", {"entities": [(27, 35, "ARRIVER"), (39, 48, "DEPART")]}),
    ("Pouvez-vous me dire comment aller de Paris à Lille ?", {"entities": [(32, 37, "DEPART"), (41, 45, "ARRIVER")]}),
    ("Il y a-t-il des trains de Toulouse à Lyon ?", {"entities": [(26, 34, "DEPART"), (38, 42, "ARRIVER")]}),
    ("Je souhaite voyager de Marseille à Nice", {"entities": [(29, 38, "DEPART"), (42, 46, "ARRIVER")]}),
    ("Y a-t-il un moyen de se rendre de Bordeaux à Toulouse ?", {"entities": [(27, 35, "DEPART"), (39, 47, "ARRIVER")]}),
    ("Je veux aller voir ma famille à Lyon en partant de Paris", {"entities": [(27, 35, "ARRIVER"), (39, 44, "DEPART")]}),
    ("Pouvez-vous me dire comment aller de Lille à Marseille ?", {"entities": [(32, 37, "DEPART"), (41, 50, "ARRIVER")]}),
    ("Il y a-t-il des trains de Strasbourg à Lyon ?", {"entities": [(26, 38, "DEPART"), (42, 46, "ARRIVER")]}),
    ("Je veux aller passer mes vacances à Paris en partant de Bordeaux", {"entities": [(27, 35, "ARRIVER"), (39, 48, "DEPART")]}),
    ("Pouvez-vous me dire comment aller de Nice à Marseille ?", {"entities": [(32, 36, "DEPART"), (40, 49, "ARRIVER")]}),
    ("Il y a-t-il des trains de Toulouse à Bordeaux ?", {"entities": [(26, 34, "DEPART"), (38, 45, "ARRIVER")]}),
    ("Je souhaite voyager de Lyon à Strasbourg", {"entities": [(29, 34, "DEPART"), (38, 48, "ARRIVER")]}),
    ("Y a-t-il un moyen de se rendre de Marseille à Lille ?", {"entities": [(29, 38, "DEPART"), (42, 47, "ARRIVER")]}),
    ("Je veux aller voir ma famille à Bordeaux en partant de Paris", {"entities": [(27, 35, "ARRIVER"), (39, 44, "DEPART")]}),
    ("Pouvez-vous me dire comment aller de Toulouse à Nice ?", {"entities": [(32, 37, "DEPART"), (41, 49, "ARRIVER")]}),
    ("Il y a-t-il des trains de Strasbourg à Paris ?", {"entities": [(26, 38, "DEPART"), (42, 46, "ARRIVER")]}),
    ("Je veux aller passer mes vacances à Marseille en partant de Lille", {"entities": [(27, 35, "ARRIVER"), (39, 48, "DEPART")]}),
    ("Pouvez-vous me dire comment aller de Bordeaux à Toulouse ?", {"entities": [(32, 37, "DEPART"), (41, 50, "ARRIVER")]}),
    ("Il y a-t-il des trains de Nice à Lyon ?", {"entities": [(26, 30, "DEPART"), (34, 45, "ARRIVER")]}),
    ("Je souhaite voyager de Paris à Lille", {"entities": [(29, 34, "DEPART"), (38, 43, "ARRIVER")]}),
    ("Y a-t-il un moyen de se rendre de Marseille à Lyon ?", {"entities": [(29, 38, "DEPART"), (42, 46, "ARRIVER")]}),
    ("Je veux aller voir ma famille à Toulouse en partant de Bordeaux", {"entities": [(27, 35, "ARRIVER"), (39, 44, "DEPART")]}),
    ("Pouvez-vous me dire comment aller de Lyon à Nice ?", {"entities": [(32, 37, "DEPART"), (41, 45, "ARRIVER")]}),
    ("Il y a-t-il des trains de Strasbourg à Marseille ?", {"entities": [(26, 38, "DEPART"), (42, 51, "ARRIVER")]}),
    ("Je veux aller passer mes vacances à Paris en partant de Lille", {"entities": [(27, 35, "ARRIVER"), (39, 48, "DEPART")]}),
    ("Pouvez-vous me dire comment aller de Bordeaux à Lyon ?", {"entities": [(32, 37, "DEPART"), (41, 50, "ARRIVER")]}),
    ("Il y a-t-il des trains de Nice à Toulouse ?", {"entities": [(26, 30, "DEPART"), (34, 45, "ARRIVER")]}),
    ("Je souhaite voyager de Marseille à Paris", {"entities": [(29, 38, "DEPART"), (42, 47, "ARRIVER")]}),
    ("Y a-t-il un moyen de se rendre de Toulouse à Bordeaux ?", {"entities": [(29, 38, "DEPART"), (42, 50, "ARRIVER")]}),
    ("Je veux aller voir ma famille à Lyon en partant de Paris", {"entities": [(27, 35, "ARRIVER"), (39, 44, "DEPART")]}),
]


In [34]:
import spacy
from spacy.training.example import Example
from spacy.util import minibatch, compounding
from pathlib import Path
import random

TRAIN_DATA = [
    ("Je voudrais aller de Paris à Marseille", {"entities": [(18, 23, "DEPART"), (27, 36, "ARRIVER")]}),
    # ... (your other training examples)
]

def check_alignment(nlp, train_data):
    for text, annotations in train_data:
        doc = nlp.make_doc(text)
        tags = spacy.training.offsets_to_biluo_tags(doc, annotations.get("entities", []))
        print(f"Text: {text}\nTags: {tags}\n")

def train_ner(model, new_model_name, output_dir, n_iter, train_data):
    # Load the pre-trained spaCy model
    nlp = spacy.load(model)

    # Vérification de la présence du composant NER
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner)
        print("ADD PIPE")
    else:
        ner = nlp.get_pipe("ner")
        print("GET PIPE")
    
    # Supprimer les autres composants du pipeline
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    
    # Ajouter ou mettre à jour les labels du NER
    for label in ["DEPART", "ARRIVER"]:
        ner.add_label(label)

    # Initialisation de l'optimiseur
    if model:
        optimizer = nlp.resume_training()
    else:
        optimizer = nlp.begin_training()

    # Remove other pipeline components
    with nlp.disable_pipes(*other_pipes):
        # Check entity alignment
        check_alignment(nlp, train_data)

        # Train the NER model
        for itn in range(n_iter):
            random.shuffle(train_data)
            losses = {}

            for batch in minibatch(train_data, size=compounding(1.0, 4.0, 1.001)):
                examples = []
                for text, annotations in batch:
                    example = Example.from_dict(nlp.make_doc(text), annotations)
                    examples.append(example)

                nlp.update(examples, drop=0.35, losses=losses, sgd=optimizer)

            # print("Iteration {} - Losses: {}".format(itn, losses))

    # Save the trained model
    if output_dir:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta["name"] = new_model_name
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

    # Test the model
    test_ner(output_dir, "Je voudrais aller de Paris à Marseille")

def test_ner(model, test_text):
    # Load the trained spaCy model
    nlp = spacy.load(model)

    # Process the test text
    doc = nlp(test_text)

    # Display detected entities with their labels
    print("\nEntities in '{}':".format(test_text))
    for ent in doc.ents:
        print(ent.label_, ent.type_, ent.text)

if __name__ == "__main__":
    train_ner(model="fr_core_news_sm", new_model_name="model_test_train_1", output_dir="./model/model_test_train_1", n_iter=1000, train_data=TRAIN_DATA)


GET PIPE
Text: Je voudrais aller de Paris à Marseille
Tags: ['O', 'O', 'O', '-', '-', '-', '-']



/Users/jlconsulting/Documents/Epitech/T-AIA-901-NLP/AIA/lib/python3.10/site-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Je voudrais aller de Paris à Marseille" with entities "[(18, 23, 'DEPART'), (27, 36, 'ARRIVER')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Saved model to model/model_test_train_1

Entities in 'Je voudrais aller de Paris à Marseille':


AttributeError: 'spacy.tokens.span.Span' object has no attribute 'type_'